In [1]:
from flask import Flask
app = Flask(__name__)

# Spring Controller와 동일한 역할 수행
@app.route("/")
def hello(): # hello 함수를 실행한다.
    return "<h1>Hello Python</h1>"

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Sep/2021 15:43:40] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Sep/2021 15:43:41] "GET /favicon.ico HTTP/1.1" 404 -


In [ ]:
# 1) https://developers.kakao.com/ 로그인

# 2) 내 어플리케이션 -> test(이전에 만들었던 사항) 클릭

# 3) 동의항목 카테고리 클릭 : 닉네임 - 필수동의, 카카오계정(이메일) - 선택동의, 카톡메세지 - 선택동의
                            # 동의 목적 - Kakao API 테스트

# 4) 카카오 로그인 -> 활성화 ON, Redirect URI -> https://localhost.com 입력

# 5) 내 어플리케이션 > 앱설정 > 플랫폼 -> Web 사이트 도메인 입력 : https://www.naver.com, https://localhost.com

# 6) Kakao API 문서 위치 - https://developers.kakao.com/docs/latest/ko/kakaologin/rest-api -> oauth 검색

# 7) 토큰값(주소) 발급받기 : 시크릿 창 모드를 사용해야한다. 안그러면 해킹당할 위험 높음.
          # kauth.kakao.com/oauth/authorize?client_id={REST_API_KEY}&redirect_uri={REDIRECT_URI}&response_type=code
          # {REST_API_KEY} = 나의 REST API 키, {REDIRECT_URI} = https://localhost.com

# 8) 해당 URL로 진입하면 (사이트에 연결할 수 없음 창이 뜬다. 정상이다. 여기서 주소에 주어진 토큰값을 사용한다.) 
                      # Kakao API를 활용할 수 있는 토큰을 발급 받을 수 있다. 주어진 시간내에 처리 하지 않으면 토큰이 만료됨.


In [16]:
# 9) 토큰값 스프링과 연동
import requests

url = "https://kauth.kakao.com/oauth/token"

# data = {
#    "grant_type" : "authorization_code",
#    "client_id" : "REST_API_키",
#    "redirect_uri" : "https://localhost.com",
#    "code" : "발급받은 토큰값(주소)"
# }

data = {
    "grant_type" : "authorization_code",
    "client_id" : "보안보안보안보안",
    "redirect_uri" : "https://localhost.com",
    "code" : "cp72xoHfnfdyl5Q2vygMGg6G8Bh9cxEMZl2XNkywNYbePXOPm-Od6u3TffuppkaHiHGGOwo9cuoAAAF8MMNH8w"
}

response = requests.post(url, data=data)

if response.status_code != 200:
    print("Error!")
else:
    tokens = response.json()
    print(tokens)

{'access_token': '2A9Hi4Dq-_gfmY6fk6Gj_5QgiGaOcObIOvxwGAopb9QAAAF8MMQLXA', 'token_type': 'bearer', 'refresh_token': '3IkDP4G8POOONGOMahekoUIE3QqGKqWRLePS0Qopb9QAAAF8MMQLWg', 'expires_in': 21599, 'scope': 'account_email talk_message profile_nickname', 'refresh_token_expires_in': 5183999}


In [19]:
import json
import requests
import datetime
import os

# Kakao Token 저장 파일
KAKAO_TOKEN_SAVED_FILE = "res/kakao_msg/kakao_token.json"

# 저장
def save_tokens(filename, tokens):
    with open(filename, "w") as fp:
        json.dump(tokens, fp)
        
# 읽기
def load_tokens(filename):
    with open(filename) as fp:
        tokens = json.load(fp)
        
    return tokens

# Access Token 갱신을 위한 Refresh
def update_tokens(app_key, filename):
    tokens = load_tokens(filename)
    
    url = "https://kauth.kakao.com/oauth/token"
    
    data = {
        "grant_type" : "refresh_token",
        "client_id" : app_key,
        "refresh_token" : tokens['refresh_token']
    }
    
    response = requests.post(url, data=data)
    
    if response.status_code != 200:
        print("Error!")
        tokens = None
    else:
        print(response.json())
        
        # 기존 내용 백업
        now = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        backup_filename = filename + "," + now
        os.rename(filenam, backup_filename)
        
        # 새롭게 업데이트된 토큰 저장
        tokens['access_token'] = response.json()['access_token']
        save_tokens(filename, tokens)
        
    return tokens

# token 저장
save_tokens(KAKAO_TOKEN_SAVED_FILE, tokens)

In [23]:
# Kakao Talk Message 보내기
# 1) https://developers.kakao.com/docs/latest/ko/message/rest-api

import json
import requests

# 토큰 정보 활용
tokens = load_tokens(KAKAO_TOKEN_SAVED_FILE)

# 메시지 보내기 URL
url = "https://kapi.kakao.com/v2/api/talk/memo/default/send"

# requests parameter 설정
headers = {
    # Authorization: Bearer {ACCESS_TOKEN}
    "Authorization" : "Bearer " + tokens['access_token']
}

data = {
    "template_object" : json.dumps({
        "object_type" : "text",
        "text" : "배송이 도착했습니다.",
        "link" : {
            # 이동할 링크 정보니까 사이트 주소를 걸면 더 좋음
            "web_url" : "www.naver.com"
        }
    })
}

# 메시지 전송
response = requests.post(url, headers=headers, data=data)
print(response.status_code)

if response.status_code != 200:
    print("Error!")
else:
    print("카톡 메시지 전송!")

200
카톡 메시지 전송!


In [25]:
from flask import Flask, url_for
app = Flask(__name__)

# Spring Controller와 동일한 역할 수행
@app.route("/")
def hello(): # hello 함수를 실행한다.
    return "<h1>Hello Python</h1>"

@app.route("/member/<id>") # id = 가변 인자
def get_member(id):
    return 'member : ' + id

if __name__ == "__main__":
    with app.test_request_context():
        print(url_for('hello'))
        print(url_for('get_member', id='yoonyeong'))

/
/member/yoonyeong


In [31]:
# Cors 문제 발생함(Vue, React 혹은 Svelte 통신사)
from flask import Flask, url_for, request
app = Flask(__name__)

# Spring Controller와 동일한 역할 수행
@app.route("/")
def hello(): # hello 함수를 실행한다.
    return "<h1>Hello Python</h1>"

@app.route("/member/<id>") # id = 가변 인자
def get_member(id):
    return 'member : ' + id

@app.route("/test", methods=['POST', 'GET'])
def chk_get_post(entity=None): # 값이 아무것도 안들어오면 디폴드로 None 세팅
    err = None
    
    if request.method == 'POST':
        id = request.form['id']
        email = request.form['email']
        
        if not id and not email:
            return add_entity(requests.form)
    else:
        err = "Invalid!!!"
        
    return err

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Sep/2021 19:18:35] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Sep/2021 19:18:39] "GET /test HTTP/1.1" 200 -
127.0.0.1 - - [29/Sep/2021 19:18:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Sep/2021 19:18:51] "GET /test HTTP/1.1" 200 -


In [ ]:
# Cors 문제 해결
# pip install flask_cors
from flask import Flask, url_for, request, jsonify
from flask_cors import CORS, cross_origin

app = Flask(__name__)
CORS(app)

# Spring Controller와 동일한 역할 수행
@app.route("/")
def hello(): # hello 함수를 실행한다.
    return "<h1>Hello Python</h1>"

@app.route("/member/<id>") # id = 가변 인자
def get_member(id):
    return 'member : ' + id

@app.route("/login", methods=['POST', 'GET'])
def chk_get_post(entity=None): # 값이 아무것도 안들어오면 디폴드로 None 세팅
    err = None
    
    if request.method == 'POST':
        print("POST 처리")
        # userId = request.form['id']
        # pw = request.form['pw']
        # print("userId = {0}, pw = {1}",format(userId, pw))
        # request.get_json() 상대편에서 보낸 json 내용을 읽음
        data = request.get_json()
        print(data)
        
        # 여기에 카톡 Open API 연동하거나 딥러닝을 활용하는 것임
        # 물론 우리는 시간이 부족해서 딥러닝이 불가함
        # 그러므로 카톡 Open API나 주식 API 등등 여러가지 Open API들을 활용하면 될 것임
        
        # return "userId = " + userId + ", password = " + pw
        # json 데이터를 전송
        return jsonify(data)
    else:
        print("GET 처리")
        err = "Invalid!!!"
        
    return err

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Sep/2021 23:44:41] "OPTIONS /login HTTP/1.1" 200 -
127.0.0.1 - - [29/Sep/2021 23:44:42] "POST /login HTTP/1.1" 200 -


POST 처리
{'userId': 'test', 'password': 'test'}
